# 参考资料

- [python登录豆瓣并获取自己看过的电影 - Johu - 博客园](https://www.cnblogs.com/johu/p/12970233.html)

# 导入相关的包

In [1]:
import requests
from pyquery import PyQuery  
import re
import csv

# 登录

In [2]:
url_basic = 'https://accounts.douban.com/j/mobile/login/basic'

In [9]:
ua_headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
    "Cookie": '',
}
   
data = {
    'ck': '',
    'name': '',
    'password': '',
    'remember': 'true',
    'ticket': ''
    }


In [10]:
s = requests.Session()
    
r = s.post(url=url_basic, data=data, headers=ua_headers)

In [11]:
r.text

'{"r": 1, "code": 403, "message": "Access denied"}'

# 构造url

In [ ]:
id_list =["hatashi","mlln"]

In [2]:
book_read_url = "https://book.douban.com/people/{user}/collect?start={num}"


In [22]:
test_url = book_read_url.format(user="hatashi",num=0)

# 请求网页

In [ ]:
login_
session = requests.Session()
login_data = {
    'username': '',
    'password': ''
}

# 登录
session.post(login_url, data=login_data)

In [25]:
def get_html(url):         # 添加：定义函数名
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/119.0.0.0 Safari/537.36'}
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html            # 添加：返回网址的html数据

In [26]:
test_html = get_html(test_url)

In [27]:
test_html

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n'

# 解析网页

In [16]:
# 函数：解析数据  extract_bookinfo_list(html)
# 参数说明：html为网页源代码的字符串
# 返回值：bookinfo_list为书籍的字典列表
def extract_bookinfo_list(html):         # 添加：定义函数名
    bookinfo_list = []
    doc = PyQuery(html)
    for book in doc.items('div.info'):
        book_name = book('h2 a').text() #书名   
        comments = book('div.short-note p').text() #短评
        rating_raw = book('div.short-note div span:first').attr("class") #评分
        rating = re.findall('\d',rating_raw)
        info_list = book('div.pub').text().split('/')   #书籍信息 
        publisher = info_list[-3].strip()   
        pub_time = info_list[-2]
        price = info_list[-1]
        authors = ''.join(info_list[:-3])
        date_raw = book('span.date').text()       
        date = re.findall('[0-9]+-[0-9]+-[0-9]+', date_raw)[0]

        bookinfo = {'book_name':book_name,
                    'authors':authors,
                    'publisher':publisher,
                    'pub_time':pub_time,
                    'comments':comments,
                    'rating':rating,
                    'date':date,
                    'price':price,
                    }

        bookinfo_list.append(bookinfo)

        
    return bookinfo_list               # 添加：返回书籍的字典列表

In [21]:
doc =PyQuery(test_html)
doc

for book in doc.items("div.info"):
    print(book("h2 a").text())

In [17]:
extract_bookinfo_list(test_html)

[]